In [1]:
#imports
import lightgbm as lgb
import numpy as np
import pandas as pd
import sqlalchemy
from catboost import CatBoostRegressor
from hyperopt import hp
from hyperopt.pyll import scope
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from catboost import CatBoostRegressor

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from models.hyperopt_model import HyperoptModel
from models.utils import CustomDataFrameMapper
from models.utils import _tree_features_transformations

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# read the data
preprocessed_data = pd.read_pickle('./data/preprocesed_data.200.20.big.pd')
preprocessed_data.columns

# Create a dataframe
df = pd.DataFrame(preprocessed_data)


In [3]:
# Create two new dataframes, one with the training rows, one with the test rows
train, test=train_test_split(df, test_size=0.2, random_state=0)

In [4]:
# Show the number of observations for the test and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 114411
Number of observations in the test data: 28603


In [5]:
    #some parameters
    DO_LOWESS = False
    hyperopt = True
    # Run CatBoostRegressor with hyperopt optimization
    answer_features_transformations, \
    question_features_transformations, \
    time_features_transformations, \
    user_features_transformations = _tree_features_transformations()
    
    cat = HyperoptModel(train.copy(), test.copy(), 'cat', cv=3)
    cat.raw_features = []
    cat.pipeline = Pipeline([
        ('prepare_features', FeatureUnion([
            ('user_features', CustomDataFrameMapper(user_features_transformations)),
            ('time_features', CustomDataFrameMapper(time_features_transformations)),
            ('answer_features', CustomDataFrameMapper(answer_features_transformations)),
            ('question_features', CustomDataFrameMapper(question_features_transformations))
        ])),
        ('estimate', CatBoostRegressor(**{'bagging_temperature': 0.29793733267072053,
                                          'iterations': 5000,
                                          'l2_leaf_reg': 1.5511980979084095,
                                          'learning_rate': 0.08683045812519587,
                                          'loss_function': 'RMSE',
                                          'random_seed': 0,
                                          'random_strength': 20,
                                          'verbose': True,
                                          'train_dir': 'outputs/cat'}))
    ])
    
    for transformer in cat.pipeline.named_steps['prepare_features'].transformer_list:
        cat.raw_features += [t[0] if isinstance(t[0], str) else t[0][0] for t in transformer[1].features]
    
    cat.space = {
        'estimate__iterations': hp.choice('estimate__iterations', [50]),
        'estimate__loss_function': hp.choice('estimate__loss_function', ['RMSE']),
        'estimate__train_dir': hp.choice('estimate__train_dir', ['outputs/cat']),
        'estimate__thread_count': hp.choice('estimate__thread_count', [4]),
        'estimate__used_ram_limit': hp.choice('estimate__used_ram_limit', [1024 * 1024 * 1024 * 4]),  # 4gb
        'estimate__random_seed': hp.choice('estimate__random_seed', [0]),

        'estimate__learning_rate': hp.loguniform('estimate__learning_rate', -5, 0),
        'estimate__random_strength': hp.choice('estimate__random_strength', [1, 20]),
        'estimate__l2_leaf_reg': hp.loguniform('estimate__l2_leaf_reg', 0, np.log(10)),
        'estimate__bagging_temperature': hp.uniform('estimate__bagging_temperature', 0, 1),
    }

    """
    find the best number of trees (following https://tech.yandex.com/catboost/doc/dg/concepts/parameter-tuning_trees-number-docpage/#parameter-tuning_trees-number)
    need to also substitute the CatBoostRegressor parameters with:
        ('estimate', CatBoostRegressor(iterations=10000, loss_function='RMSE', auto_stop_pval=1e-4, use_best_model=True, train_dir='outputs/cat_trees', verbose=True))
    """
    # num_trees_train, num_trees_eval = train_test_split(train, test_size=0.2, random_state=0)
    # X = num_trees_train[list(filter(lambda column: column in cat.raw_features, cat.train.columns))]
    # y = num_trees_train['score']
    #
    # eval_X = num_trees_eval[list(filter(lambda column: column in cat.raw_features, cat.train.columns))]
    # eval_X = cat.pipeline.named_steps['prepare_features'].fit_transform(eval_X)
    # eval_set = (eval_X, num_trees_eval['score'])
    #
    # cat.pipeline.fit(X=X, y=y, estimate__eval_set=eval_set)
    # cat.model = cat.pipeline
    # print(cat.model.named_steps['estimate'].get_params())
    # cat.model.named_steps['estimate'].save_model('tmp/cat.model'.encode('utf-8'))

    if hyperopt:
        cat.run(do_lowess=DO_LOWESS)
    else:
        # train with default params
        cat.pipeline.fit(X=cat.X_train, y=cat.y_train)
        cat.model = cat.pipeline
        cat.stats()

        # joblib.dump(cat, './outputs/models/%s.pckl' % cat.output_prefix)
        # cat = joblib.load('./outputs/models/%s.pckl' % cat.output_prefix)

        cat.plot_predicted_vs_actual(do_lowess=False)
        cat.plot_residuals(r_type='raw', do_lowess=False)
        cat.plot_feature_importance()

Performing parameters optimization...
Borders generated
0:	learn 0.5075347432passed: 0.179 sec	total: 851ms	remaining: 41.7s
1:	learn 0.4690903129passed: 0.192 sec	total: 1.04s	remaining: 25s
2:	learn 0.4337510891passed: 0.252 sec	total: 1.29s	remaining: 20.3s
3:	learn 0.4012527298passed: 0.188 sec	total: 1.48s	remaining: 17.1s
4:	learn 0.3716233459passed: 0.197 sec	total: 1.68s	remaining: 15.1s
5:	learn 0.344565189passed: 0.151 sec	total: 1.83s	remaining: 13.4s
6:	learn 0.3198474292passed: 0.182 sec	total: 2.02s	remaining: 12.4s
7:	learn 0.2971846859passed: 0.144 sec	total: 2.16s	remaining: 11.3s
8:	learn 0.2763446443passed: 0.134 sec	total: 2.29s	remaining: 10.4s
9:	learn 0.257178628passed: 0.139 sec	total: 2.43s	remaining: 9.73s
10:	learn 0.2397118277passed: 0.134 sec	total: 2.57s	remaining: 9.11s
11:	learn 0.2239045409passed: 0.151 sec	total: 2.72s	remaining: 8.61s
12:	learn 0.2094209738passed: 0.134 sec	total: 2.85s	remaining: 8.12s
13:	learn 0.1965322164passed: 0.132 sec	total: 2

18:	learn 0.1481003884passed: 0.152 sec	total: 3.57s	remaining: 5.83s
19:	learn 0.1411624381passed: 0.143 sec	total: 3.71s	remaining: 5.57s
20:	learn 0.1349674989passed: 0.153 sec	total: 3.87s	remaining: 5.34s
21:	learn 0.1294309637passed: 0.149 sec	total: 4.02s	remaining: 5.11s
22:	learn 0.1245683379passed: 0.141 sec	total: 4.16s	remaining: 4.88s
23:	learn 0.1203173708passed: 0.152 sec	total: 4.31s	remaining: 4.67s
24:	learn 0.1164641802passed: 0.148 sec	total: 4.46s	remaining: 4.46s
25:	learn 0.1130902349passed: 0.144 sec	total: 4.6s	remaining: 4.25s
26:	learn 0.1100701386passed: 0.152 sec	total: 4.76s	remaining: 4.05s
27:	learn 0.1073856468passed: 0.169 sec	total: 4.93s	remaining: 3.87s
28:	learn 0.1050329154passed: 0.169 sec	total: 5.09s	remaining: 3.69s
29:	learn 0.1030668099passed: 0.161 sec	total: 5.26s	remaining: 3.5s
30:	learn 0.1012678273passed: 0.152 sec	total: 5.41s	remaining: 3.31s
31:	learn 0.09974941133passed: 0.145 sec	total: 5.55s	remaining: 3.12s
32:	learn 0.098407263

37:	learn 0.09388130173passed: 0.136 sec	total: 6.39s	remaining: 2.02s
38:	learn 0.0932885884passed: 0.149 sec	total: 6.54s	remaining: 1.84s
39:	learn 0.09278804718passed: 0.177 sec	total: 6.71s	remaining: 1.68s
40:	learn 0.09232313823passed: 0.179 sec	total: 6.89s	remaining: 1.51s
41:	learn 0.0919358256passed: 0.157 sec	total: 7.05s	remaining: 1.34s
42:	learn 0.09157752817passed: 0.152 sec	total: 7.2s	remaining: 1.17s
43:	learn 0.0912858343passed: 0.153 sec	total: 7.36s	remaining: 1s
44:	learn 0.09100330256passed: 0.147 sec	total: 7.5s	remaining: 834ms
45:	learn 0.09075477182passed: 0.153 sec	total: 7.66s	remaining: 666ms
46:	learn 0.0904853079passed: 0.156 sec	total: 7.81s	remaining: 499ms
47:	learn 0.09026839269passed: 0.15 sec	total: 7.96s	remaining: 332ms
48:	learn 0.09005727241passed: 0.161 sec	total: 8.13s	remaining: 166ms
49:	learn 0.08987425748passed: 0.147 sec	total: 8.27s	remaining: 0us
Borders generated
0:	learn 0.5074671183passed: 0.139 sec	total: 795ms	remaining: 38.9s
1:

4:	learn 0.09402242898passed: 0.151 sec	total: 1.54s	remaining: 13.8s
5:	learn 0.09374007482passed: 0.147 sec	total: 1.68s	remaining: 12.4s
6:	learn 0.09345419393passed: 0.147 sec	total: 1.83s	remaining: 11.3s
7:	learn 0.09318112301passed: 0.142 sec	total: 1.97s	remaining: 10.4s
8:	learn 0.09296036595passed: 0.138 sec	total: 2.11s	remaining: 9.63s
9:	learn 0.09256299372passed: 0.139 sec	total: 2.25s	remaining: 9.01s
10:	learn 0.09245585791passed: 0.147 sec	total: 2.4s	remaining: 8.51s
11:	learn 0.09236830277passed: 0.148 sec	total: 2.55s	remaining: 8.07s
12:	learn 0.09207573375passed: 0.155 sec	total: 2.7s	remaining: 7.7s
13:	learn 0.09183357253passed: 0.142 sec	total: 2.85s	remaining: 7.32s
14:	learn 0.09141038379passed: 0.143 sec	total: 2.99s	remaining: 6.98s
15:	learn 0.09096025172passed: 0.151 sec	total: 3.14s	remaining: 6.67s
16:	learn 0.09050809539passed: 0.154 sec	total: 3.29s	remaining: 6.4s
17:	learn 0.09024599192passed: 0.139 sec	total: 3.44s	remaining: 6.11s
18:	learn 0.0898

22:	learn 0.08821511309passed: 0.145 sec	total: 4.13s	remaining: 4.85s
23:	learn 0.08779516265passed: 0.159 sec	total: 4.29s	remaining: 4.64s
24:	learn 0.08755135337passed: 0.142 sec	total: 4.43s	remaining: 4.43s
25:	learn 0.08725876532passed: 0.151 sec	total: 4.58s	remaining: 4.23s
26:	learn 0.08701153114passed: 0.145 sec	total: 4.73s	remaining: 4.03s
27:	learn 0.08686291653passed: 0.148 sec	total: 4.88s	remaining: 3.83s
28:	learn 0.08663413596passed: 0.144 sec	total: 5.02s	remaining: 3.63s
29:	learn 0.08644448456passed: 0.144 sec	total: 5.16s	remaining: 3.44s
30:	learn 0.08629881102passed: 0.153 sec	total: 5.32s	remaining: 3.26s
31:	learn 0.08611786035passed: 0.144 sec	total: 5.46s	remaining: 3.07s
32:	learn 0.08596620877passed: 0.154 sec	total: 5.62s	remaining: 2.89s
33:	learn 0.08577904224passed: 0.141 sec	total: 5.76s	remaining: 2.71s
34:	learn 0.08556753869passed: 0.146 sec	total: 5.9s	remaining: 2.53s
35:	learn 0.08542759841passed: 0.149 sec	total: 6.05s	remaining: 2.35s
36:	lea

...

36:	learn 0.0841527903passed: 0.138 sec	total: 5.72s	remaining: 2.01s
37:	learn 0.0840175288passed: 0.13 sec	total: 5.85s	remaining: 1.85s
38:	learn 0.08396723654passed: 0.133 sec	total: 5.99s	remaining: 1.69s
39:	learn 0.08384698807passed: 0.133 sec	total: 6.12s	remaining: 1.53s
40:	learn 0.08374398756passed: 0.134 sec	total: 6.25s	remaining: 1.37s
41:	learn 0.08365393395passed: 0.13 sec	total: 6.38s	remaining: 1.22s
42:	learn 0.08349256582passed: 0.132 sec	total: 6.51s	remaining: 1.06s
43:	learn 0.08340943605passed: 0.128 sec	total: 6.64s	remaining: 906ms
44:	learn 0.08331510079passed: 0.132 sec	total: 6.78s	remaining: 753ms
45:	learn 0.08326267494passed: 0.128 sec	total: 6.9s	remaining: 600ms
46:	learn 0.08317468027passed: 0.135 sec	total: 7.04s	remaining: 449ms
47:	learn 0.08307452458passed: 0.136 sec	total: 7.18s	remaining: 299ms
48:	learn 0.08300052285passed: 0.138 sec	total: 7.31s	remaining: 149ms
49:	learn 0.08292931518passed: 0.13 sec	total: 7.44s	remaining: 0us
Borders genera

2:	learn 0.1026368601passed: 0.138 sec	total: 1.08s	remaining: 16.9s
3:	learn 0.09412018748passed: 0.136 sec	total: 1.22s	remaining: 14s
4:	learn 0.09191484179passed: 0.139 sec	total: 1.35s	remaining: 12.2s
5:	learn 0.0908291695passed: 0.132 sec	total: 1.49s	remaining: 10.9s
6:	learn 0.09010204113passed: 0.134 sec	total: 1.62s	remaining: 9.96s
7:	learn 0.0896310882passed: 0.132 sec	total: 1.75s	remaining: 9.21s
8:	learn 0.08921245444passed: 0.134 sec	total: 1.89s	remaining: 8.6s
9:	learn 0.08898642736passed: 0.13 sec	total: 2.02s	remaining: 8.07s
10:	learn 0.08868656771passed: 0.13 sec	total: 2.15s	remaining: 7.62s
11:	learn 0.08845890524passed: 0.129 sec	total: 2.28s	remaining: 7.21s
12:	learn 0.08810135023passed: 0.137 sec	total: 2.42s	remaining: 6.87s
13:	learn 0.08782610256passed: 0.13 sec	total: 2.54s	remaining: 6.54s
14:	learn 0.08769031688passed: 0.134 sec	total: 2.68s	remaining: 6.25s
15:	learn 0.08753777261passed: 0.132 sec	total: 2.81s	remaining: 5.97s
16:	learn 0.08733086472

20:	learn 0.2001210394passed: 0.141 sec	total: 3.51s	remaining: 4.85s
21:	learn 0.1921748571passed: 0.133 sec	total: 3.65s	remaining: 4.64s
22:	learn 0.1845784162passed: 0.14 sec	total: 3.79s	remaining: 4.45s
23:	learn 0.1775232728passed: 0.129 sec	total: 3.92s	remaining: 4.24s
24:	learn 0.170944248passed: 0.133 sec	total: 4.05s	remaining: 4.05s
25:	learn 0.1646831352passed: 0.135 sec	total: 4.19s	remaining: 3.86s
26:	learn 0.1589082187passed: 0.135 sec	total: 4.32s	remaining: 3.68s
27:	learn 0.1534860936passed: 0.135 sec	total: 4.46s	remaining: 3.5s
28:	learn 0.1484438653passed: 0.136 sec	total: 4.59s	remaining: 3.33s
29:	learn 0.1437869571passed: 0.129 sec	total: 4.72s	remaining: 3.15s
30:	learn 0.1393982266passed: 0.138 sec	total: 4.86s	remaining: 2.98s
31:	learn 0.135410107passed: 0.131 sec	total: 4.99s	remaining: 2.81s
32:	learn 0.1316239187passed: 0.138 sec	total: 5.13s	remaining: 2.64s
33:	learn 0.1282288244passed: 0.129 sec	total: 5.26s	remaining: 2.48s
34:	learn 0.1250139752pa

38:	learn 0.1145005995passed: 0.139 sec	total: 5.97s	remaining: 1.68s
39:	learn 0.1123531554passed: 0.135 sec	total: 6.11s	remaining: 1.53s
40:	learn 0.1104292453passed: 0.141 sec	total: 6.25s	remaining: 1.37s
41:	learn 0.1086747347passed: 0.131 sec	total: 6.38s	remaining: 1.22s
42:	learn 0.1070575849passed: 0.139 sec	total: 6.52s	remaining: 1.06s
43:	learn 0.1055485971passed: 0.133 sec	total: 6.65s	remaining: 907ms
44:	learn 0.104204883passed: 0.133 sec	total: 6.79s	remaining: 754ms
45:	learn 0.1029596076passed: 0.131 sec	total: 6.92s	remaining: 601ms
46:	learn 0.1017806018passed: 0.14 sec	total: 7.06s	remaining: 450ms
47:	learn 0.1007099669passed: 0.132 sec	total: 7.19s	remaining: 300ms
48:	learn 0.09975296703passed: 0.138 sec	total: 7.33s	remaining: 150ms
49:	learn 0.09885333014passed: 0.133 sec	total: 7.46s	remaining: 0us
Borders generated
0:	learn 0.5219995511passed: 0.138 sec	total: 821ms	remaining: 40.2s
1:	learn 0.495812212passed: 0.133 sec	total: 954ms	remaining: 22.9s
2:	lear

5:	learn 0.09046703161passed: 0.212 sec	total: 2.14s	remaining: 15.7s
6:	learn 0.08991493026passed: 0.212 sec	total: 2.36s	remaining: 14.5s
7:	learn 0.089459053passed: 0.208 sec	total: 2.56s	remaining: 13.5s
8:	learn 0.08916117018passed: 0.204 sec	total: 2.77s	remaining: 12.6s
9:	learn 0.088767223passed: 0.21 sec	total: 2.98s	remaining: 11.9s
10:	learn 0.08852826359passed: 0.208 sec	total: 3.19s	remaining: 11.3s
11:	learn 0.08810794051passed: 0.229 sec	total: 3.42s	remaining: 10.8s
12:	learn 0.08778221717passed: 0.244 sec	total: 3.66s	remaining: 10.4s
13:	learn 0.08754213738passed: 0.222 sec	total: 3.88s	remaining: 9.98s
14:	learn 0.0873333713passed: 0.202 sec	total: 4.09s	remaining: 9.53s
15:	learn 0.08713158881passed: 0.208 sec	total: 4.29s	remaining: 9.12s
16:	learn 0.08694662214passed: 0.21 sec	total: 4.5s	remaining: 8.74s
17:	learn 0.08683953117passed: 0.206 sec	total: 4.71s	remaining: 8.37s
18:	learn 0.08669269786passed: 0.215 sec	total: 4.93s	remaining: 8.04s
19:	learn 0.0865734